In [1]:
import gc
import os
import pickle
import sys

from typing import Tuple
from datetime import datetime
from pathlib import Path

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import torch

from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
data_path = Path.cwd().parent / "data"
stock_data_path = data_path / "stock_data"
image_file_path = data_path / "stock_data" / "stock_chart_image"

In [3]:
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:',device)

True
학습을 진행하는 기기: cuda:0


In [4]:
size = 64

In [5]:
def read_img(img_path, size):
    img = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (size, size))
    img = Image.fromarray(img)
    # img = img.resize((size, size))
    label = int(img_path.split("_")[-1][0])
    
    return img, label

In [6]:
X = []
Y = []
i = 0
for img_path in tqdm(os.listdir(image_file_path)[270000:275000]):
    if img_path == ".ipynb_checkpoints":
        continue
    img, label = read_img(str(image_file_path / img_path), size)
    if label == 0:
        continue
    X.append(img)
    Y.append(0 if label == 1 else 1)
    # X.append(img)
    # Y.append(label)
    
X = np.asarray(X)
Y = np.asarray(Y)

with open("X_data.pkl", "wb") as f:
    pickle.dump(X, f)
    
with open("Y_data.pkl", "wb") as f:
    pickle.dump(Y, f)

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:45<00:00, 109.69it/s]
/tmp/ipykernel_5559/3228630649.py:15: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  X = np.asarray(X)
/tmp/ipykernel_5559/3228630649.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)


In [5]:
with open("X_data.pkl", "rb") as f:
    X = pickle.load(f)
    
with open("Y_data.pkl", "rb") as f:
    Y = pickle.load(f)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y)

In [8]:
import torch
import torchvision.transforms

# transform = torchvision.transforms.Compose(
#     [torchvision.transforms.Resize(size, size),
#      torchvision.transforms.ToTensor(),
#      torchvision.transforms.Normalize(mean=[meanR, meanG, meanB], std=[stdR, stdG, stdB])])

class Dataset(torch.utils.data.Dataset):
    def __init__(self, 
                X_data: np.ndarray,
                Y_data: np.ndarray):
        super().__init__()
        
        self.X_data = X_data
        self.Y_data = Y_data
        self.len = len(Y_data)
        # self.meanRGB, self.stdRGB = self._calculate_mean_std()
        self.transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor()
                # torchvision.transforms.Normalize(mean=self.meanRGB, std=self.stdRGB)
            ]
        )
        
    def __getitem__(self, idx: int) -> Tuple:
        return self.transform(self.X_data[idx]), self.Y_data[idx]
        
    def __len__(self):
        return self.len
    
    def _calculate_mean_std(self):
        transform = torchvision.transforms.Compose(
            [torchvision.transforms.ToTensor()])
        meanRGB = [np.mean(transform(x).numpy(), axis=(1,2)) for x in self.X_data]
        stdRGB = [np.std(transform(x).numpy(), axis=(1,2)) for x in self.X_data]

        meanR = np.mean([m[0] for m in meanRGB])
        meanG = np.mean([m[1] for m in meanRGB])
        meanB = np.mean([m[2] for m in meanRGB])

        stdR = np.mean([s[0] for s in stdRGB])
        stdG = np.mean([s[1] for s in stdRGB])
        stdB = np.mean([s[2] for s in stdRGB])
        
        return (meanR, meanG, meanB), (stdR, stdG, stdB)
        

In [9]:
trainset = Dataset(X_train, Y_train)
testset = Dataset(X_test, Y_test)

In [10]:
batch_size = 32

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class BaseModel(nn.Module):
    
    def training_step(self, batch, criterion):
        images, labels = batch 
        images = images.to(device)
        labels = labels.to(device)
        
        out = self(images)                  # Generate predictions
        loss = criterion(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch, criterion):
        images, labels = batch 
        images = images.to(device)
        labels = labels.to(device)
        
        out = self(images)                    # Generate predictions
        loss = criterion(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [12]:
class VGG16(BaseModel):
    def __init__(self):
        super().__init__()
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.conv4_1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(int(size * size / 2), 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 2)

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5) #dropout was included to combat overfitting
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.5)
        x = self.fc3(x)
        return x

In [13]:
from torch.optim.lr_scheduler import _LRScheduler

class ExponentialLR(_LRScheduler):
    def __init__(self, optimizer, end_lr, num_iter, last_epoch=-1):
        self.end_lr = end_lr
        self.num_iter = num_iter
        super(ExponentialLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        curr_iter = self.last_epoch
        r = curr_iter / self.num_iter
        return [base_lr * (self.end_lr / base_lr) ** r for base_lr in
                self.base_lrs]

In [14]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  
@torch.no_grad()
def evaluate(model, val_loader, criterion):
    model.eval()
    outputs = [model.validation_step(batch, criterion) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, criterion, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(), lr)
    # lr_scheduler = ExponentialLR(optimizer, 10, 100)
    
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for batch in tqdm(train_loader):
            optimizer.zero_grad() 
            loss = model.training_step(batch, criterion)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            
            # lr_scheduler.step()
            
        result = evaluate(model, val_loader, criterion)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
        
        if epoch == 0 or history[-1]["val_loss"] < min([result["val_loss"] for result in history[:-1]]):
            torch.save(model.state_dict(), "model.pt")
            print("Successfully saved model")
    
    return history

In [15]:
model = VGG16()
model.to(device)

VGG16(
  (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_3): Conv2d(512, 51

In [16]:
model.load_state_dict(torch.load("model.pt"))

<All keys matched successfully>

In [34]:
model = VGG16()
# model.load_state_dict(torch.load("model.pt"))
model.to(device)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
num_epochs = 100
opt_func = torch.optim.Adam
lr = 1e-7
history = fit(num_epochs, lr, model, trainloader, testloader, criterion, opt_func)

100%|███████████████████████████████████████████████████████████████████████████████████| 59/59 [00:08<00:00,  7.16it/s]


Epoch [0], train_loss: 1.1000, val_loss: 1.0987, val_acc: 0.3131
Successfully saved model


100%|███████████████████████████████████████████████████████████████████████████████████| 59/59 [00:08<00:00,  7.25it/s]


Epoch [1], train_loss: 1.0982, val_loss: 1.0970, val_acc: 0.4011
Successfully saved model


100%|███████████████████████████████████████████████████████████████████████████████████| 59/59 [00:08<00:00,  7.28it/s]


...

Epoch [97], train_loss: 0.8394, val_loss: 0.8394, val_acc: 0.6846


100%|███████████████████████████████████████████████████████████████████████████████████| 59/59 [00:08<00:00,  6.70it/s]


Epoch [98], train_loss: 0.8399, val_loss: 0.8427, val_acc: 0.6846


100%|███████████████████████████████████████████████████████████████████████████████████| 59/59 [00:09<00:00,  6.12it/s]


Epoch [99], train_loss: 0.8398, val_loss: 0.8399, val_acc: 0.6846


In [17]:
@torch.no_grad()
def eval(model, batch):
    model.eval()
    images, labels = batch
    images = images.to(device)
    labels = labels.to(device)
    dummy = torch.randn((10, 3, 64, 64)).to(device)
    out = model(images)
    pred = torch.max(out, 1)[1]
    print(pred)
    print(labels)
    print(pred == labels)

In [32]:
iterator = iter(trainloader)
eval(model, iterator.next())

tensor([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1], device='cuda:0')
tensor([1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        1, 0, 1, 0, 1, 1, 1, 0], device='cuda:0')
tensor([ True, False, False,  True, False, False, False, False,  True,  True,
        False,  True,  True,  True, False, False, False,  True,  True, False,
        False,  True,  True, False,  True, False,  True, False,  True, False,
         True, False], device='cuda:0')


In [68]:
def save_chart_image(df:pd.DataFrame, 
                     save_path: Path,
                     file_prefix: str,
                     window=20):
    customstyle = mpf.make_mpf_style(base_mpf_style='yahoo', facecolor='w')

    width_config={"candle_linewidth":1.5, 
                  "candle_width":0.9, 
                  "volume_width": 0.6, 
                  "line_width": 1}
                     
    for i in tqdm(range(len(df) - window - 5), desc=f"Data {file_prefix}"):
        target_df = df[i: i + window]
        label = label_data(df, i + window - 1, 5, 0.04)
        
        ma_5 = mpf.make_addplot(target_df.ma_5)
        ma_20 = mpf.make_addplot(target_df.ma_20)
        ma_60 = mpf.make_addplot(target_df.ma_60)
        # ma_120 = mpf.make_addplot(target_df.ma_120)
        # ma_240 = mpf.make_addplot(target_df.ma_240)
        
        file_path = save_path / f"{file_prefix}_{i}_{label}.jpg"

        fig = mpf.plot(target_df, type="candle",
                       style=customstyle, 
                         addplot=[ma_5, ma_20, ma_60],
                         update_width_config=width_config, 
                         figsize=(10, 10),
                         fontscale=0,
                         axisoff=True,
                         volume=True,
                         tight_layout=True,
                         savefig=file_path)